In [26]:
import scala.util.Random
import java.time._

import scala.util.Random
import java.time._


In [27]:
val start = LocalDateTime.of(2021, 1, 1, 12, 0).toEpochSecond(ZoneOffset.UTC)

start: Long = 1609502400


In [28]:
val randomDateTime = LocalDateTime.ofEpochSecond(start + Random.nextInt(86400*365), 5000, ZoneOffset.UTC)

randomDateTime: java.time.LocalDateTime = 2021-07-17T12:44:24.000005


In [29]:
def randomTemperatureGenerator(): String = {
    if(Random.nextFloat()<.2) {
        return "NaN"
    }
    BigDecimal((50 - Random.nextInt(100)) * Random.nextFloat()).setScale(6, BigDecimal.RoundingMode.HALF_UP).toString
}

randomTemperatureGenerator: ()String


In [30]:
def randomLocationGenerator(): String = {
    if(Random.nextFloat()<.2) {
        return "NaN"
    }
    val latitude = BigDecimal((90 - Random.nextInt(180)) * Random.nextFloat()).setScale(6, BigDecimal.RoundingMode.HALF_UP).toString
    val longitude = BigDecimal((180 - Random.nextInt(360)) * Random.nextFloat()).setScale(6, BigDecimal.RoundingMode.HALF_UP).toString
    latitude+","+longitude
}

randomLocationGenerator: ()String


In [31]:
var countriesCodeList = java.util.Locale.getISOCountries()

countriesCodeList: Array[String] = Array(AD, AE, AF, AG, AI, AL, AM, AO, AQ, AR, AS, AT, AU, AW, AX, AZ, BA, BB, BD, BE, BF, BG, BH, BI, BJ, BL, BM, BN, BO, BQ, BR, BS, BT, BV, BW, BY, BZ, CA, CC, CD, CF, CG, CH, CI, CK, CL, CM, CN, CO, CR, CU, CV, CW, CX, CY, CZ, DE, DJ, DK, DM, DO, DZ, EC, EE, EG, EH, ER, ES, ET, FI, FJ, FK, FM, FO, FR, GA, GB, GD, GE, GF, GG, GH, GI, GL, GM, GN, GP, GQ, GR, GS, GT, GU, GW, GY, HK, HM, HN, HR, HT, HU, ID, IE, IL, IM, IN, IO, IQ, IR, IS, IT, JE, JM, JO, JP, KE, KG, KH, KI, KM, KN, KP, KR, KW, KY, KZ, LA, LB, LC, LI, LK, LR, LS, LT, LU, LV, LY, MA, MC, MD, ME, MF, MG, MH, MK, ML, MM, MN, MO, MP, MQ, MR, MS, MT, MU, MV, MW, MX, MY, MZ, NA, NC, NE, NF, NG, NI, NL, NO, NP, NR, NU, NZ, OM, PA, PE, PF, PG, PH, PK, PL, PM, PN, PR, PS, PT, PW, PY, QA, RE, RO, ...


In [38]:
def randomCountryCodeGenerator(): String = {
    if(Random.nextFloat()<.2) {
        return "NaN"
    }
    countriesCodeList(Random.nextInt(countriesCodeList.length))
}

randomCountryCodeGenerator: ()String


In [39]:
val df = spark.sparkContext.parallelize(
            Seq.fill(20){
                (LocalDateTime.ofEpochSecond(start + Random.nextInt(86400*365), 5000,ZoneOffset.UTC).toString
                 , randomTemperatureGenerator()
                 , randomLocationGenerator()
                 , randomCountryCodeGenerator()
            )}, 10).toDF("timestamp", "temperature", "location", "observatory")

df: org.apache.spark.sql.DataFrame = [timestamp: string, temperature: string ... 2 more fields]


In [40]:
df.show()

+--------------------+-----------+--------------------+-----------+
|           timestamp|temperature|            location|observatory|
+--------------------+-----------+--------------------+-----------+
|2021-07-02T04:57:...|   0.000000|  7.037897,89.790657|         KE|
|2021-01-27T19:44:...|  11.254812|-65.456100,45.934063|         LK|
|2021-07-22T05:57:...|   9.144113|-7.730503,-74.412445|         MH|
|2021-10-12T13:31:...| -28.528172|                 NaN|         MK|
|2021-09-08T06:54:...|        NaN|  8.562175,74.383644|        NaN|
|2021-01-17T19:35:...|   8.163269|-14.027365,-128.6...|         VI|
|2021-05-01T09:59:...|   3.016435|-27.532867,22.758667|         PR|
|2021-12-24T20:36:...|   7.473855| 42.925018,16.073051|         GH|
|2021-02-16T04:36:...|        NaN|-28.753862,-6.613416|         BH|
|2021-01-29T23:15:...|  -3.113942|                 NaN|        NaN|
|2021-11-07T20:33:...|  35.118458|-74.940826,63.670044|         AF|
|2021-05-15T12:27:...|  14.567891|-10.101830,-44

In [41]:
df.write.option("header","true").mode("overwrite").csv("weatherbaloon.csv")